In [ ]:
'''
Sources:
HDI
- Human Development Index (HDI).csv:
http://hdr.undp.org/en/data
- human-development-index-escosura.csv:
https://ourworldindata.org/human-development-index

GINI
- economic-inequality-gini-index.csv:
https://ourworldindata.org/grapher/economic-inequality-gini-index

shapefile
- Countries_WGS84.shp:
https://hub.arcgis.com/datasets/a21fdb46d23e4ef896f31475217cbb08_1

https://www.naturalearthdata.com/downloads/10m-cultural-vectors/
'''

# GINI Index

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [ ]:
df_map = gpd.read_file('./geo/Countries_WGS84/Countries_WGS84.shp')
df_map.columns = ['country_code', 'country', 'geometry']
df_map.drop(df_map.index[14], inplace=True) #Drop row corresponding to 'Antarctica'

In [ ]:
df_gini = pd.read_csv('./data/GINI/economic-inequality-gini-index.csv')

In [ ]:
df_gini_pivot = df_gini.pivot_table(index=['Entity', 'Code'], columns='Year', values='GINI index (World Bank estimate)').reset_index()
df_gini_pivot = df_gini_pivot.fillna(method='bfill', axis=1)
df_gini_pivot = df_gini_pivot.fillna(method='ffill', axis=1)
#df_gini_pivot.fillna(0, inplace=True)
df_gini_pivot

In [ ]:
df_gini_pivot[df_gini_pivot['Entity'] == 'Denmark'] 

In [ ]:
df_gini_pivot[df_gini_pivot['Entity'] == 'Brazil'] 

In [ ]:
replace_values  = {
     'Myanmar':'Myanmar (Burma)',
     'Saint Lucia':'St. Lucia',
     'Tanzania':'Tanzania, United Republic of'
    }

df_gini_pivot.replace({'Entity': replace_values}, inplace=True)

In [ ]:
YEAR = 2011
gpd_gini = pd.merge(df_map, df_gini_pivot[['Entity', YEAR]], left_on='country', right_on='Entity', how = 'left')
gpd_gini[[YEAR]] = gpd_gini[[YEAR]].fillna(0)

In [ ]:
import json
from bokeh.io import output_notebook, show, output_file
from bokeh.models import LogColorMapper, LinearColorMapper, ColorBar, LogTicker, GeoJSONDataSource
from bokeh.palettes import brewer, d3, all_palettes
from bokeh.plotting import figure

In [ ]:
geosource = GeoJSONDataSource(geojson = gpd_gini.to_json())

In [ ]:
#Define a sequential multi-hue color palette.
palette = all_palettes['Viridis'][8]
#Reverse color order
palette = palette[::-1]

color_mapper = LinearColorMapper(palette=palette, low = 0, high = gpd_gini[YEAR].max())

TOOLS = "pan,wheel_zoom,reset,hover,save"

tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=8,
                     width = 500,
                     height = 20,
                     border_line_color=None,
                     location = (0,0), 
                     orientation = 'horizontal')
                     #major_label_overrides = tick_labels)

p = figure(
    title='GINI Index, ' + str(YEAR), 
    tools=TOOLS,
    plot_height = 480,
    plot_width = 720,
    x_axis_location=None,
    y_axis_location=None,
    tooltips=[
        ("Country", "@country"), ("GINI", "@"+str(YEAR))
    ])
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

p.patches('xs','ys', source=geosource,
          fill_color={'field': str(YEAR), 'transform': color_mapper},
          fill_alpha=0.8, line_color="black", line_width=0.2)

p.add_layout(color_bar, 'below')

output_notebook()

show(p)

# HDI 

In [ ]:
df_hdi = pd.read_csv('./data/HDI/Human Development Index (HDI).csv', sep=',', skiprows=1)
df_hdi.dropna(axis=1, how='all', inplace=True) # drop empty columns
df_hdi.drop(df_hdi.index[189:], inplace=True) # drop last part of the table
df_hdi.replace('..', 0, inplace=True) # replace null values to zero
df_hdi.iloc[:,2:] = df_hdi.iloc[:,2:].astype(float)

In [ ]:
replace_values  = {
     'Bahamas':'Bahamas, The',
     'Belarus':'Byelarus',
     'Bolivia (Plurinational State of)':'Bolivia',
     'Brunei Darussalam':'Brunei',
     'Cabo Verde':'Cape Verde',
     'Czechia':'Czech Republic',
     'Gambia':'Gambia, The',
     'Iran (Islamic Republic of)':'Iran',
     'Korea (Republic of)':'South Korea',
     'Micronesia (Federated States of)':'Federated States of Micronesia',
     'Moldova (Republic of)':'Moldova',
     'Myanmar':'Myanmar (Burma)',
     'North Macedonia':'Macedonia',
     'Palau':'Pacific Islands (Palau)',
     'Russian Federation':'Russia',
     'Saint Kitts and Nevis':'St. Kitts and Nevis',
     'Saint Lucia':'St. Lucia',
     'Saint Vincent and the Grenadines':'St. Vincent and the Grenadines',
     'Samoa':'Western Samoa',
     'Syrian Arab Republic':'Syria',
     'Tanzania (United Republic of)':'Tanzania, United Republic of',
     'Venezuela (Bolivarian Republic of)':'Venezuela',
     'Viet Nam':'Vietnam'
    }

df_hdi.replace({'Country': replace_values}, inplace=True)

In [ ]:
df_hdi[df_hdi['Country'] == 'Denmark'] 

In [ ]:
df_hdi[df_hdi['Country'] == 'Brazil'] 

In [ ]:
YEAR = str(YEAR)
gpd_hdi = pd.merge(df_map, df_hdi[['Country', YEAR]], left_on='country', right_on='Country', how = 'left')
gpd_hdi[[YEAR]] = gpd_hdi[[YEAR]].fillna(0)

In [ ]:
geosource = GeoJSONDataSource(geojson = gpd_hdi.to_json())

In [ ]:
#Define a sequential multi-hue color palette.
palette = all_palettes['Viridis'][8]
#Reverse color order 
#palette = palette[::-1]

color_mapper = LinearColorMapper(palette=palette, low = 0, high = 1)

TOOLS = "pan,wheel_zoom,reset,hover,save"

tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=8,
                     width = 500,
                     height = 20,
                     border_line_color=None,
                     location = (0,0), 
                     orientation = 'horizontal')
                     #major_label_overrides = tick_labels)

p = figure(
    title='Human Development Index, ' + YEAR, 
    tools=TOOLS,
    plot_height = 480,
    plot_width = 720,
    x_axis_location=None,
    y_axis_location=None,
    tooltips=[
        ("Country", "@country"), ("HDI", "@"+YEAR)
    ])
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

p.patches('xs','ys', source=geosource,
          fill_color={'field': YEAR, 'transform': color_mapper},
          fill_alpha=0.8, line_color="black", line_width=0.1)

p.add_layout(color_bar, 'below')

output_notebook()

show(p)

# Normalising Data 

In [ ]:
YEAR = int(YEAR)

In [ ]:
# GINI normalized
df_gini_pivot['norm_'+str(YEAR)] = ((df_gini_pivot[YEAR]-df_gini_pivot[YEAR].min())/(df_gini_pivot[YEAR].max()-df_gini_pivot[YEAR].min()))

In [ ]:
# Inverted GINI
df_gini_pivot['inv_'+str(YEAR)] = df_gini_pivot.loc[:,'norm_'+str(YEAR)].apply(lambda x : 1-x)

In [ ]:
# HDI normalized
df_hdi['norm_'+str(YEAR)]=((df_hdi[str(YEAR)]-df_hdi[str(YEAR)].min())/(df_hdi[str(YEAR)].max()-df_hdi[str(YEAR)].min()))

# Correlation

In [ ]:
df_corr = pd.merge(df_gini_pivot[['Entity', 'inv_'+str(YEAR)]], df_hdi[['Country', 'norm_'+str(YEAR)]], left_on='Entity', right_on='Country')
df_corr['corr'] = df_corr['inv_'+str(YEAR)] - df_corr['norm_'+str(YEAR)]

In [ ]:
df_corr

In [ ]:
gpd_corr = pd.merge(df_map, df_corr[['Country', 'corr']], left_on='country', right_on='Country', how = 'left')
gpd_corr[['corr']] = gpd_corr[['corr']].fillna(0)

In [ ]:
geosource = GeoJSONDataSource(geojson = gpd_corr.to_json())

In [ ]:
#Define a sequential multi-hue color palette.
palette = all_palettes['Viridis'][8]
#Reverse color order 
#palette = palette[::-1]

color_mapper = LinearColorMapper(palette=palette, low = -1, high = 1)

TOOLS = "pan,wheel_zoom,reset,hover,save"

tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=8,
                     width = 500,
                     height = 20,
                     border_line_color=None,
                     location = (0,0), 
                     orientation = 'horizontal')
                     #major_label_overrides = tick_labels)

p = figure(
    title='GINI Index vs Human Development Index, ' + str(YEAR), 
    tools=TOOLS,
    plot_height = 480,
    plot_width = 720,
    x_axis_location=None,
    y_axis_location=None,
    tooltips=[
        ("Country", "@country"), ("Correl", "@corr")
    ])
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

p.patches('xs','ys', source=geosource,
          fill_color={'field': 'corr', 'transform': color_mapper},
          fill_alpha=0.8, line_color="black", line_width=0.1)

p.add_layout(color_bar, 'below')

output_notebook()

show(p)